# Guidelines

In [1]:
# reading in the data via the Kaggle API

# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# install Kaggle
! pip install kaggle

In [3]:
!mkdir ~/.kaggle

In [4]:

!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c detecting-french-texts-difficulty-level-2023

100% 303k/303k [00:00<00:00, 638kB/s]
100% 303k/303k [00:00<00:00, 637kB/s]


In [6]:


from zipfile import ZipFile
with ZipFile('detecting-french-texts-difficulty-level-2023.zip','r') as zip:
  zip.extractall(path="")

In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv('training_data.csv', index_col = 'id')

In [8]:
df.head()

,sentence,difficulty
id,,
0,Les coûts kilométriques réels peuvent diverger...,C1
1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,Le test de niveau en français est sur le site ...,A1
3,Est-ce que ton mari est aussi de Boston?,A1
4,"Dans les écoles de commerce, dans les couloirs...",B1


In [9]:
df_pred = pd.read_csv('unlabelled_test_data.csv', index_col = 'id')
df_pred.head()

,sentence
id,
0,Nous dûmes nous excuser des propos que nous eû...
1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,"Et, paradoxalement, boire froid n'est pas la b..."
3,"Ce n'est pas étonnant, car c'est une saison my..."
4,"Le corps de Golo lui-même, d'une essence aussi..."


In [10]:
df_example_submission = pd.read_csv('sample_submission.csv', index_col = 'id')
df_example_submission.head()

,difficulty
id,
0,A1
1,A1
2,A1
3,A1
4,A1


# Maintenant on code

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder


df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_features=5000)

X_train_tfidf = vectorizer.fit_transform(df_train['sentence'])
X_test_tfidf = vectorizer.transform(df_test['sentence'])

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(df_train['difficulty'])
y_test_encoded = label_encoder.transform(df_test['difficulty'])

model = LogisticRegression(C=1.0)
model.fit(X_train_tfidf, y_train_encoded)

predictions = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test_encoded, predictions)
print("Accuracy:", accuracy)

precision = precision_score(y_test_encoded, predictions, average='weighted')
print("Precision:", precision)

recall = recall_score(y_test_encoded, predictions, average='weighted')
print("Recall:", recall)

f1 = f1_score(y_test_encoded, predictions, average='weighted')
print("F1-score:", f1)



Accuracy: 0.4583333333333333
Precision: 0.45118514649757796
Recall: 0.4583333333333333
F1-score: 0.4494501949238123


In [12]:

X_pred_tfidf = vectorizer.transform(df_pred['sentence'])


predictions_pred = model.predict(X_pred_tfidf)

predicted_difficulty = label_encoder.inverse_transform(predictions_pred)

df_pred['difficulty'] = predicted_difficulty

print(df_pred[['sentence', 'difficulty']])

                                               sentence difficulty
id                                                                
0     Nous dûmes nous excuser des propos que nous eû...         C2
1     Vous ne pouvez pas savoir le plaisir que j'ai ...         A2
2     Et, paradoxalement, boire froid n'est pas la b...         A1
3     Ce n'est pas étonnant, car c'est une saison my...         A1
4     Le corps de Golo lui-même, d'une essence aussi...         C2
...                                                 ...        ...
1195  C'est un phénomène qui trouve une accélération...         B1
1196  Je vais parler au serveur et voir si on peut d...         A2
1197  Il n'était pas comme tant de gens qui par pare...         C2
1198      Ils deviennent dangereux pour notre économie.         B2
1199  Son succès a généré beaucoup de réactions néga...         B2

[1200 rows x 2 columns]


In [13]:
df_pred.drop('sentence',  axis=1, inplace=True)

df_pred


,difficulty
id,
0,C2
1,A2
2,A1
3,A1
4,C2
...,...
1195,B1
1196,A2
1197,C2


In [14]:
#df_pred.to_csv('submission.csv')

#! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "UNIL_Rolex"